 # Experiment 1

In [1]:
import preprocessors

In [56]:
myLoader = preprocessors.Loader("C:/Users/emily/Documents/DissertationProject/training/training_setA_csv", ['HR', 'O2Sat', 'SBP', 'MAP', 'DBP', 'Resp', 'Temp'])
df = myLoader.ARIMA_VAR_preprocessor(max_patients=100, min_time_points=30)
y_train, y_pred = myLoader.split_train_test(df, 4)

Patient 0: Dropping — constant columns found
Patient 1: Less than 30 time points — skipping.
Patient 3: Less than 30 time points — skipping.
Patient 4: Dropping — constant columns found
Patient 5: Less than 30 time points — skipping.
Patient 9: Less than 30 time points — skipping.
Patient 10: Dropping — constant columns found
Patient 11: Less than 30 time points — skipping.
Patient 12: Dropping — constant columns found
Patient 14: Less than 30 time points — skipping.
Patient 15: Less than 30 time points — skipping.
Patient 16: Dropping — constant columns found
Patient 19: Less than 30 time points — skipping.
Patient 21: Less than 30 time points — skipping.
Patient 22: Less than 30 time points — skipping.
Patient 23: Dropping — constant columns found
Patient 24: Dropping — constant columns found
Patient 25: Less than 30 time points — skipping.
Patient 26: Less than 30 time points — skipping.
Patient 31: Dropping — constant columns found
Patient 32: Dropping — constant columns found
Pati

In [3]:
from sktime.forecasting.var import VAR
model = VAR(maxlags=2, method="ols", verbose=False, trend="c", random_state=42)

In [4]:
from sktime.forecasting.base import ForecastingHorizon

model.fit(y_train)
fh = ForecastingHorizon([1, 2, 3, 4], is_relative=True)
forecasts = model.predict(fh)

In [40]:
forecasts

HR      O2Sat         SBP        MAP        DBP  \
Patient_ID ICULOS                                                            
0          44       70.862070  96.606026  157.424137  90.406187  66.518350   
           45       72.017566  96.317298  158.910113  94.720804  67.250587   
           46       73.806358  96.417051  155.880152  91.899776  65.550761   
           47       75.167536  96.795875  152.375212  89.790486  63.889030   
1          41      114.962771  95.330566  120.420020  77.046488  60.547877   
...                       ...        ...         ...        ...        ...   
95         39       97.861972  94.923259  134.879332  73.246784  50.416845   
96         42       51.904165  99.574414  111.460142  71.652546  51.378877   
           43       52.478270  99.004143  109.313918  71.538120  51.436881   
           44       53.659174  99.055318  108.509953  70.462749  50.314254   
           45       53.473427  99.256590  112.129461  72.187269  51.088954   

                        Resp       Temp  
Patient_ID ICULOS                        
0          44      23.382905  37.260062  
           45      23.742392  37.458839  
           46      26.959493  38.118423  
           47      27.697126  38.359054  
1          41      21.363237  37.715698  
...                      ...        ...  
95         39      30.909908  36.574575  
96         42       8.972587  37.373600  
           43      10.294684  37.213147  
           44      11.060529  37.051725  
           45      11.408650  37.018478  

[388 rows x 7 columns]

In [7]:
import evaluators

myEvaluator = evaluators.Evaluator(y_train, y_pred, forecasts)
results = myEvaluator.sktime_dtw()
myEvaluator.sktime_ddtw()
myEvaluator.tslearn_dtw()

Mean sktime DTW distance for feature HR: 430.6348299632063
Mean sktime DTW distance for feature O2Sat: 26.052426743937513
Mean sktime DTW distance for feature SBP: 1470.487551996165
Mean sktime DTW distance for feature MAP: 665.5541113093424
Mean sktime DTW distance for feature DBP: 406.39751812593914
Mean sktime DTW distance for feature Resp: 75.4094156327051
Mean sktime DTW distance for feature Temp: 1.0890900583583811
Mean sktime DDTW distance for feature HR: 70.45827378496547
Mean sktime DDTW distance for feature O2Sat: 5.119314403816615
Mean sktime DDTW distance for feature SBP: 366.73642064963514
Mean sktime DDTW distance for feature MAP: 178.2568025041841
Mean sktime DDTW distance for feature DBP: 116.4563631465721
Mean sktime DDTW distance for feature Resp: 23.702112021211086
Mean sktime DDTW distance for feature Temp: 0.11503470668182936
Mean tslearn DTW distance: 47.05199057386343


In [8]:
results

{'HR': [13.273968130431648,
  418.426544197115,
  218.08693054783419,
  13.241392711404771,
  369.5594931884913,
  243.13133701586236,
  442.5139947320913,
  1004.5131882786076,
  171.04176358199058,
  34.92698741451563,
  67.18422198084465,
  609.2920626461853,
  4.445996596548629,
  84.31635567173835,
  120.1195677132832,
  50.1529835081316,
  139.2809259851718,
  18.82192063540446,
  121.16535501711925,
  74.24807203201462,
  447.9593230264758,
  80.28873771124753,
  425.0599319523681,
  220.9774880030123,
  430.96123853433744,
  87.33755278043887,
  120.9042758721952,
  7.950698346778779,
  184.96442597744556,
  96.31436860707882,
  101.86571164087522,
  0.4059655352257865,
  407.72701993238894,
  65.89956521269511,
  937.6205529533578,
  41.48721971842389,
  7612.324723569871,
  59.69434368478146,
  181.9546348216137,
  252.56603880616274,
  354.40513432001825,
  1024.2817036388276,
  79.06952013913458,
  46.99054898364637,
  258.442125128381,
  658.6405277911317,
  254.7406695257

# Experiment 2

In [9]:
model2 = VAR(random_state=42)
model2.fit(y_train)
fh2 = ForecastingHorizon([1, 2, 3, 4], is_relative=True)
forecasts2 = model2.predict(fh2)

In [11]:
import evaluators

evaluator2 = evaluators.Evaluator(y_train, y_pred, forecasts2)
results2 = evaluator2.sktime_dtw()
evaluator2.sktime_ddtw()
evaluator2.tslearn_dtw()

Mean sktime DTW distance for feature HR: 257.4799977564295
Mean sktime DTW distance for feature O2Sat: 16.32944891659957
Mean sktime DTW distance for feature SBP: 855.1139090585318
Mean sktime DTW distance for feature MAP: 385.5281223655886
Mean sktime DTW distance for feature DBP: 249.7784873763099
Mean sktime DTW distance for feature Resp: 65.03943137637346
Mean sktime DTW distance for feature Temp: 0.691052382422895
Mean sktime DDTW distance for feature HR: 39.916085192384024
Mean sktime DDTW distance for feature O2Sat: 3.011179169199121
Mean sktime DDTW distance for feature SBP: 225.81735498428506
Mean sktime DDTW distance for feature MAP: 115.40163376393292
Mean sktime DDTW distance for feature DBP: 60.166069918443775
Mean sktime DDTW distance for feature Resp: 17.589264726257966
Mean sktime DDTW distance for feature Temp: 0.06616687509990911
Mean tslearn DTW distance: 38.62906845055766


# Experiment 3

In [12]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.var import VAR
from sktime.transformations.series.difference import Differencer

model3 = TransformedTargetForecaster(steps=[
    ("diff", Differencer()),     # makes the data stationary
    ("var", VAR(random_state=42))               # fit VAR on differenced data
])

In [13]:
model3.fit(y_train)
fh3 = ForecastingHorizon([1, 2, 3, 4], is_relative=True)
forecasts3 = model3.predict(fh3)

In [14]:
import evaluators
evaluator3 = evaluators.Evaluator(y_train, y_pred, forecasts3)
results3 = evaluator3.sktime_dtw()
evaluator3.sktime_ddtw()
evaluator3.tslearn_dtw()

Mean sktime DTW distance for feature HR: 307.06367570369036
Mean sktime DTW distance for feature O2Sat: 21.72496480165652
Mean sktime DTW distance for feature SBP: 1078.720209805275
Mean sktime DTW distance for feature MAP: 617.3957353430585
Mean sktime DTW distance for feature DBP: 302.9825536266127
Mean sktime DTW distance for feature Resp: 81.9503431738789
Mean sktime DTW distance for feature Temp: 0.7796419474319193
Mean sktime DDTW distance for feature HR: 47.03408803279134
Mean sktime DDTW distance for feature O2Sat: 3.319857766420785
Mean sktime DDTW distance for feature SBP: 221.7489548457172
Mean sktime DDTW distance for feature MAP: 110.73247950604583
Mean sktime DDTW distance for feature DBP: 52.223335271401666
Mean sktime DDTW distance for feature Resp: 19.760665274763856
Mean sktime DDTW distance for feature Temp: 0.07398809408861437
Mean tslearn DTW distance: 41.733895116954784


# Experiment 4

In [25]:
import evaluators
evaluator4 = evaluators.Evaluator(y_train, y_pred, forecasts4)
evaluator4.sktime_dtw()
evaluator4.sktime_ddtw()
evaluator4.tslearn_dtw()


Mean sktime DTW distance for feature HR: 257.4799977564295
Mean sktime DTW distance for feature O2Sat: 16.32944891659957
Mean sktime DTW distance for feature SBP: 855.1139090585318
Mean sktime DTW distance for feature MAP: 385.5281223655886
Mean sktime DTW distance for feature DBP: 249.7784873763099
Mean sktime DTW distance for feature Resp: 65.03943137637346
Mean sktime DTW distance for feature Temp: 0.691052382422895
Mean sktime DDTW distance for feature HR: 39.916085192384024
Mean sktime DDTW distance for feature O2Sat: 3.011179169199121
Mean sktime DDTW distance for feature SBP: 225.81735498428506
Mean sktime DDTW distance for feature MAP: 115.40163376393292
Mean sktime DDTW distance for feature DBP: 60.166069918443775
Mean sktime DDTW distance for feature Resp: 17.589264726257966
Mean sktime DDTW distance for feature Temp: 0.06616687509990911
Mean tslearn DTW distance: 38.62906845055766


In [38]:

from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.split import ExpandingWindowSplitter
from sktime.forecasting.naive import NaiveForecaster


cv = ExpandingWindowSplitter(initial_window=5, step_length=1, fh=[1,2,3])
forecaster = NaiveForecaster()
param_grid = {"strategy": ["last", "mean", "median"]}
gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid=param_grid,
    cv=cv)
gscv.fit(y_train)


ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=[1, 2, 3],
                                                   initial_window=5),
                        forecaster=NaiveForecaster(),
                        param_grid={'strategy': ['last', 'mean', 'median']})

In [39]:
fcsts = gscv.predict(fh)

In [41]:
gscv.best_params_

{'strategy': 'last'}

In [42]:
gscv.forecaster

NaiveForecaster()

In [43]:
gscv.best_score_

0.07327360655949282

In [44]:
gscv.cv_results_

,mean_test_MeanAbsolutePercentageError,mean_fit_time,mean_pred_time,params,rank_test_MeanAbsolutePercentageError
0,0.073274,0.370095,0.812796,{'strategy': 'last'},1.0
1,0.087780,0.409763,0.269577,{'strategy': 'mean'},2.0
2,NaN,NaN,NaN,{'strategy': 'median'},NaN


In [54]:
from sktime.forecasting.var import VAR

model = VAR(random_state=42)
from sktime.forecasting.base import ForecastingHorizon

model.fit(y_train)
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6], is_relative=True)
forecasts = model.predict(fh)

In [55]:
import evaluators
myEvaluator = evaluators.Evaluator(y_train, y_pred, forecasts)
results = myEvaluator.sktime_dtw()
myEvaluator.sktime_ddtw()
myEvaluator.tslearn_dtw()

Mean sktime DTW distance for feature HR: 418.87475856769316
Mean sktime DTW distance for feature O2Sat: 23.793561398470764
Mean sktime DTW distance for feature SBP: 1623.159462608799
Mean sktime DTW distance for feature MAP: 862.9895467823309
Mean sktime DTW distance for feature DBP: 397.54697256130555
Mean sktime DTW distance for feature Resp: 97.08166936176546
Mean sktime DTW distance for feature Temp: 1.359576606252328
Mean sktime DDTW distance for feature HR: 87.83126637119435
Mean sktime DDTW distance for feature O2Sat: 7.058571819729809
Mean sktime DDTW distance for feature SBP: 412.77137532054553
Mean sktime DDTW distance for feature MAP: 236.0506662480186
Mean sktime DDTW distance for feature DBP: 115.80986827330854
Mean sktime DDTW distance for feature Resp: 39.17631550517632
Mean sktime DDTW distance for feature Temp: 0.15931832972442125
Mean tslearn DTW distance: 51.7578972401431
